# Parte 3: Estudio de diferentes algoritmos de machine learning

En este notebook vamos a aplicar diferentes algoritmos para estudiar su efectividad para este problema e intentaremos predicciones para los Juegos Olímpicos de 2021.

Los algoritmos que se aplican son: 
* Matriz de confusión
* Datos de entrenamiento y datos de test
* Knn - Vecinos cercanos
* Regresión Lineal
* Regresión Logística
* Naive Bayes
* Gaussian Naive Bayes
* K-means
* Perceptron
* MLP - Multi layer perceptron
* Decision Tree
* SVM - Supervised Vector Machine
* Descenso del gradiente
* Random Forest
* ¿¿RED NEURONAL??

In [1]:
import pandas as pd

import numpy as np
from numpy import * 

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Lee el excel generado en el otro documento .ipynb
ddbb=pd.read_excel("/Github/DecatlonEstadistics/resources/data.xlsx")
del ddbb['1500m NF']

ddbb['Country']=ddbb['Country'].str.lower()

ddbb.columns = ['Position', 'Athlete', 'Age', 'Country', 'Total Points', 'Year', 'Competition', '100m', 
                 '100m Points', 'Lj', 'Lj Points', 'Sp', 'Sp Points', 'Hj', 'Hj Points', '400m', '400m Points', 
                 '110m H', '110m H Points', 'Dt', 'Dt Points', 'Pv', 'Pv Points', 'Jt', 'Jt Points', 
                 '1500m Points', '1500m']

In [3]:
pKmeans = []

for i in range(len(ddbb)):
    if(ddbb.iloc[i]['Position'] <= 10):
        pKmeans.append(10)
    elif(ddbb.iloc[i]['Position'] > 10 and ddbb.iloc[i]['Position'] <= 25):
        pKmeans.append(25)
    elif(ddbb.iloc[i]['Position'] > 25):
        pKmeans.append(30)

## 0. Métricas Algoritmos

In [34]:
def metricasAlgorithms(y_test, y_pred):
    metricas = []
    
    metricas.append(accuracy_score(y_test, y_pred))
    metricas.append(precision_score(y_test, y_pred, average='micro'))
    metricas.append(recall_score(y_test, y_pred, average='micro'))
    metricas.append(f1_score(y_test, y_pred, average='micro'))

    metricas.append(precision_score(y_test, y_pred, average='macro'))
    metricas.append(recall_score(y_test, y_pred, average='macro'))
    metricas.append(f1_score(y_test, y_pred, average='macro'))

    metricas.append(precision_score(y_test, y_pred, average='weighted'))
    metricas.append(recall_score(y_test, y_pred, average='weighted'))
    metricas.append(f1_score(y_test, y_pred, average='weighted'))
    
    return metricas

def metricasAlgorithmsRegl(X_test, y_test, regL):
    metricas = []
    
    metricas.append(regL.score(X_test,y_test))
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    metricas.append(0)
    
    return metricas

## 1. Datos de entrenamiento y datos de test.

Los datos se entrenan en función a todas las pruebas resultantes y se clasifican en función a la posición obtenida.
El 70% de los datos se utilizan para entrenamiento.
El 30% restante se utiliza para tests.

In [5]:
ddbbData = ddbb[['100m','100m Points','Lj','Lj Points','Sp','Sp Points','Hj','Hj Points','400m Points','400m',
                 '110m H','110m H Points','Dt','Dt Points','Pv','Pv Points','Jt','Jt Points','1500m Points','1500m']]

X_train, X_test, y_train, y_test = train_test_split(ddbbData, ddbb['Position'], test_size=0.3)

## 2. Vecinos Cercanos

In [21]:
def knnFunction(X_train, X_test, y_train, y_test):
    
    knn = KNeighborsClassifier(n_neighbors=5)

    #Entrena el modelo
    knn.fit(X_train, y_train)

    #Predice para los datos de test
    y_pred = knn.predict(X_test)
    
    vector = metricasAlgorithms(y_test, y_pred)
    
    return(vector)

## 3. Regresión Lineal

In [27]:
def RegLinealFunction(X_train, X_test, y_train, y_test):
    
    regL = linear_model.LinearRegression()
    
    regL.fit(X_train, y_train)
    
    y_pred = regL.predict(X_test)
    
    return metricasAlgorithmsRegl(X_test, y_test, regL)

## 4. Regresión Logística

In [8]:
def RegLogisFunction(X_train, X_test, y_train, y_test):
    
    logreg = linear_model.LogisticRegression(random_state=0)

    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)
    
    return metricasAlgorithms(y_test, y_pred)

## 5. Naive Bayes

In [9]:
def NaiveBayesFunction(X_train, X_test, y_train, y_test):
    
    gnb = GaussianNB()

    gnb.fit(X_train, y_train)

    y_pred = gnb.predict(X_test)

    metricasAlgorithms(y_test, y_pred)
    
    return metricasAlgorithms(y_test, y_pred)

##  6. K-Means

In [10]:
def KmeansFunction(pKmeans, ddbbData):

    X = scale(ddbbData)
    y = pd.DataFrame(pKmeans)

    kmeans1 = KMeans(n_clusters=1 , random_state= 5)

    kmeans1.fit(X)

    y_pred = np.choose(kmeans1.labels_,[10,25,30]).astype(np.int64)

    return metricasAlgorithms(y, y_pred)

## 7. Perceptron

In [11]:
def PerceptronFunction(X_train, X_test, y_train, y_test):
    
    percep = Perceptron(tol=1e-5, random_state=1)

    percep.fit(X_train, y_train)

    y_pred = percep.predict(X_test)

    metricasAlgorithms(y_test, y_pred)
    
    return metricasAlgorithms(y_test, y_pred)

## 8. MLP- Perceptron Multi Capa

In [12]:
def MLPFunction(X_train, X_test, y_train, y_test):

    mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)

    mlp.fit(X_train, y_train)

    y_pred = mlp.predict(X_test)

    metricasAlgorithms(y_test, y_pred)
    
    return metricasAlgorithms(y_test, y_pred)

## 9. SVM - Supervised Vector Machine

In [13]:
def SVMFunction(X_train, X_test, y_train, y_test):

    clf = svm.SVC(kernel='linear')

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    metricasAlgorithms(y_test, y_pred)

    return metricasAlgorithms(y_test, y_pred)

## 10. Descenso del gradiente

In [14]:
def DescensoGradFunction(X_train, X_test, y_train, y_test):

    dGrad = SGDClassifier(loss="hinge", penalty="l2")

    dGrad.fit(X_train, y_train)

    y_pred = dGrad.predict(X_test)

    metricasAlgorithms(y_test, y_pred)

    return metricasAlgorithms(y_test, y_pred)

## 11. Arbol de decision

In [15]:
def ArbolDecFunction(X_train, X_test, y_train, y_test):

    clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

    clf = clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)

    metricasAlgorithms(y_test, y_pred)
    
    return metricasAlgorithms(y_test, y_pred)

## 12. Random Forest

In [16]:
def BosqueAlFunction(X_train, X_test, y_train, y_test):

    rForest=RandomForestClassifier(n_estimators=100)

    rForest.fit(X_train,y_train)

    y_pred=rForest.predict(X_test)

    metricasAlgorithms(y_test, y_pred)

    return metricasAlgorithms(y_test, y_pred)

In [33]:
knn = knnFunction(X_train, X_test, y_train, y_test)
kmeans = KmeansFunction(pKmeans, ddbbData)
RegLin = RegLinealFunction(X_train, X_test, y_train, y_test)
RegLog = RegLogisFunction(X_train, X_test, y_train, y_test)
NBayes = NaiveBayesFunction(X_train, X_test, y_train, y_test)
Percep = PerceptronFunction(X_train, X_test, y_train, y_test)
DGrad = DescensoGradFunction(X_train, X_test, y_train, y_test)
mlp = MLPFunction(X_train, X_test, y_train, y_test)
ADecis = ArbolDecFunction(X_train, X_test, y_train, y_test)
BAleatorio = BosqueAlFunction(X_train, X_test, y_train, y_test)
dataFrameDatos = pd.DataFrame({
                                'Metricas': ['Accuracy Micro','Precision Micro','Recall Micro','F1 Micro',
                                             'Precision Macro','Recall Macro','F1 Macro','Precision Weigthed',
                                             'Recall Weigthed','F1 Weigthed'],
                                'knn': knn,
                                'Regresion lineal': RegLin,
                                'Regresion logistica': RegLog,
                                'Kmeans' : kmeans,
                                'Naive bayes': NBayes,
                                'Perceptron': Percep,
                                'MLP': mlp,
                                #'SVM': SVMFunction(X_train, X_test, y_train, y_test),
                                'Descenso gradiente': DGrad,
                                'Arbol de decision': ADecis,
                                'Bosque aleatorios': BAleatorio 
                            })


ValueError: Classification metrics can't handle a mix of continuous-multioutput and continuous targets

In [18]:
dataFrameDatos

,Metricas,knn,Regresion lineal,Regresion logistica,Kmeans,Naive bayes,Perceptron,MLP,Descenso gradiente,Arbol de decision,Bosque aleatorios
0,Accuracy Micro,0.105882,0.444081,0.050980,0.469412,0.078431,0.062745,0.031373,0.043137,0.074510,0.082353
1,Precision Micro,0.105882,0.000000,0.050980,0.469412,0.078431,0.062745,0.031373,0.043137,0.074510,0.082353
2,Recall Micro,0.105882,0.000000,0.050980,0.469412,0.078431,0.062745,0.031373,0.043137,0.074510,0.082353
3,F1 Micro,0.105882,0.000000,0.050980,0.469412,0.078431,0.062745,0.031373,0.043137,0.074510,0.082353
4,Precision Macro,0.051726,0.000000,0.043717,0.156471,0.029455,0.006367,0.001012,0.008692,0.016878,0.059842
5,Recall Macro,0.069731,0.000000,0.052358,0.333333,0.045482,0.038778,0.031250,0.035863,0.056566,0.061291
6,F1 Macro,0.056632,0.000000,0.039658,0.212970,0.034057,0.009239,0.001961,0.013446,0.022075,0.058469
7,Precision Weigthed,0.078898,0.000000,0.059699,0.220347,0.053052,0.011027,0.001016,0.011319,0.029480,0.089345
8,Recall Weigthed,0.105882,0.000000,0.050980,0.469412,0.078431,0.062745,0.031373,0.043137,0.074510,0.082353
9,F1 Weigthed,0.086260,0.000000,0.046922,0.299912,0.060614,0.015902,0.001968,0.017347,0.037650,0.082911
